 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#Reference" data-toc-modified-id="Reference-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Reference</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#Code" data-toc-modified-id="Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#generate-graph" data-toc-modified-id="generate-graph-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>generate graph</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#simple-neural-network" data-toc-modified-id="simple-neural-network-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>simple neural network</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#测试前要看一下" data-toc-modified-id="测试前要看一下-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>测试前要看一下</a></span></li></ul></li></ul></li></ul></div>

# Reference

https://medium.com/@bgoncalves/network-effects-explained-pagerank-and-preferential-attachment-61fdf93d023a

# Code

## generate graph

In [3]:
import networkx as nx

In [1]:
with open("web-Stanford.txt","r") as fil:
    for _ in range(4):
        next(fil)
    data=[tuple([int(j) for j in i.strip().split()]) for i in fil]

In [4]:
G = nx.DiGraph()
G.add_edges_from(data)

pr = nx.pagerank(G, alpha=0.9)   #作为true response?

for i in pr.keys():
    G.nodes[i]['score']=pr[i]

In [12]:
G.nodes[1]['score']

3.561215144991757e-07

In [17]:
n=len(G.nodes())

## simple neural network

In [18]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf
from gcn.utils import *
from gcn.models import Model,MLP
from gcn.layers import *

import scipy.sparse as sp
import random


### 测试前要看一下
+ 之前学习率填了0.05，明显训练的时候有了误差来回变大减小又变大再减小的情况，应该要调小一点，之前作业里是0.01,我一时兴起又改大了……
+ 用networkx里的pagerank算法出来的分数是在0-1之间，所以最后的误差出来即使是0.几，也不确定是不是很好，这里是100个点做validation。
+ epoch可能也要调，学习率0.05的时候300差不多收敛了，但是不知道学习率改了之后是什么情况

In [19]:
# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('dataset', 'cora', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'  #这行应该并没有什么用
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.05, 'Initial learning rate.')
#flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 300, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
#flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
#flags.DEFINE_float('weight_decay', 0, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 100, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

In [20]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)

#--------------- Your code here --------------#

def masked_mean_square_error(preds,labels,mask):
    """L2 loss with masking."""
    loss = tf.nn.l2_loss(preds - labels)   #这里是可以考虑ReLU什么的么，还没细想，只是突然想到的可能
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


#--------------------------------------------#

def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

In [21]:
class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_mean_square_error(self.outputs, self.placeholders['labels'],
                                                 self.placeholders['labels_mask'])
#        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
#                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=FLAGS.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            sparse_inputs=True,
                                            logging=self.logging))

        self.layers.append(GraphConvolution(input_dim=FLAGS.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x,
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return self.outputs    #tf.nn.softmax(self.outputs)

In [22]:
def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    #adj_square = normalize_adj(np.power(adj, 2).tocoo())   #A^2 有空可以尝试
    return sparse_to_tuple(adj_normalized)

In [37]:
# Randomly generate graph with 500 nodes
adj = nx.adjacency_matrix(G)

# Generate feature matrix  #there is difference between different version of networkx, 这里是用networkx 2.0. 原来1.1的用法注释掉了
features = sp.vstack(list(j[1] for j in G.in_degree(G.nodes())),dtype = 'float32').tolil()
#features = sp.vstack(list(G.in_degree(G.nodes()).values()),dtype = 'float32').tolil()   

# Feed labels to 20 nodes
label = []
for i in G.nodes():
    label.append(G.nodes[i]['score'])
        
alabel = np.array(label, dtype='float64')

idx_train = range(20000)
idx_val = range(20000, 20000 + 100)

y_train = np.zeros([n, 1])   #n is number of total nodes, =281903
train_mask = np.zeros([n, ], dtype = 'bool')
for i in idx_train:
    y_train[i] = alabel[i]
    train_mask[i] = True

y_val = np.zeros([n, 1])
val_mask = np.zeros([n, ], dtype = 'bool')
for i in idx_val:
    y_val[i] = alabel[i]
    val_mask[i] = True

In [38]:
# Load data
#adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)

# Some preprocessing
features = preprocess_features(features)
if FLAGS.model == 'gcn':
    support = [preprocess_adj(adj)]
    num_supports = 1
    model_func = GCN
elif FLAGS.model == 'gcn_cheby':
    support = chebyshev_polynomials(adj, FLAGS.max_degree)
    num_supports = 1 + FLAGS.max_degree
    model_func = GCN
elif FLAGS.model == 'dense':
    support = [preprocess_adj(adj)]  # Not used
    num_supports = 1
    model_func = MLP
else:
    raise ValueError('Invalid argument for model: ' + str(FLAGS.model))

# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
sess = tf.Session()


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)

d:\py36\lib\site-packages\gcn-1.0-py3.6.egg\gcn\utils.py:97: RuntimeWarning: divide by zero encountered in power


In [39]:
# Initialize weights
sess.run(tf.global_variables_initializer())

train_loss = []
validation_loss = []
train_accuracy = []
validation_accuracy = []


# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)
    
    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    validation_loss.append(cost)
    validation_accuracy.append(acc)
    train_loss.append(outs[1])
    train_accuracy.append(outs[2])

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and validation_loss[-1] > np.mean(validation_loss[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

Epoch: 0001 train_loss= 15932818.00000 train_acc= 0.99995 val_loss= 6530353.00000 val_acc= 1.00000 time= 4.45378
Epoch: 0002 train_loss= 9549798.00000 train_acc= 0.99995 val_loss= 1582841.00000 val_acc= 1.00000 time= 4.42326
Epoch: 0003 train_loss= 2399162.00000 train_acc= 0.99995 val_loss= 5216.68408 val_acc= 1.00000 time= 4.45093
Epoch: 0004 train_loss= 1005767.43750 train_acc= 0.99995 val_loss= 798486.62500 val_acc= 1.00000 time= 4.44427
Epoch: 0005 train_loss= 1645399.37500 train_acc= 0.99995 val_loss= 2148284.25000 val_acc= 1.00000 time= 4.41487
Epoch: 0006 train_loss= 3304748.00000 train_acc= 0.99995 val_loss= 2800197.00000 val_acc= 1.00000 time= 4.40683
Epoch: 0007 train_loss= 4370506.00000 train_acc= 0.99995 val_loss= 2251100.00000 val_acc= 1.00000 time= 4.39124
Epoch: 0008 train_loss= 2369627.75000 train_acc= 0.99995 val_loss= 1616308.75000 val_acc= 1.00000 time= 4.40668
Epoch: 0009 train_loss= 1896579.75000 train_acc= 0.99995 val_loss= 668010.50000 val_acc= 1.00000 time= 4.38